In [1]:
import pandas as pd
import numpy as np

train_org = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample = pd.read_csv("sample_submission.csv")
train = train_org.drop(['OutcomeSubtype', "AnimalID"], axis='columns', inplace=False)

In [3]:
def getAge(age):
    age = age.split(" ")
    time = age[1]
    num = age[0]
    if time == "year" or time == "years":
        return float(num)
    if time == "month" or time == "months":
        return float(num) / 12
    if time == "week" or time == "weeks":
        return float(num) / (4 * 12)
    if time == "day" or time == "days":
        return float(num) / (4 * 7 * 12)
    
def nameLength(name):
    return len(name)

def simplifyColor(color):
    return color.split("/")[0].split(" ")[0]
  
def isMix(breed):
    if breed.endswith(' mix'):
        return 1
    if breed.find("/") != -1:
        return 1
    return 0

def isSpayedNeutered(sex):
    if sex.startswith("Spayed") or sex.startswith("Neutered"):
        return 1
    return 0

def simplifyBreed(breed):
    breed = breed.split("/")[0]
    if breed.endswith(' mix'):
        breed = breed[:-4]
    return breed
    
def encodeFeatures(df_in, hotEncode):
    df = df_in.copy()
    # Encode has name
    hasname = df.Name.isnull().astype(int)
    # Encode name length
    nameslength = df.Name
    nameslength[df.Name.notna()] = df.Name[df.Name.notna()].apply(nameLength)
    nameslength[df.Name.isnull()] = -1
    nameslength.name = "NamesLength"

    # Encode date
    dates = pd.to_datetime(df.DateTime)
    year = dates.dt.year
    year = year - year.min()
    year.name = "Year"
    month = dates.dt.month
    month.name = "Month"
    day = dates.dt.day
    day.name = "Day"

    weekday = dates.dt.dayofweek
    weekday.name = "Weekday"
    if hotEncode:
        weekday = pd.get_dummies(weekday, prefix="Weekday")

    # Encode age
    ages = df.AgeuponOutcome
    ages[df.AgeuponOutcome.notna()] = df.AgeuponOutcome[df.AgeuponOutcome.notna()].apply(getAge)
    mean_age = ages[df.AgeuponOutcome.notna()].mean()
    ages[df.AgeuponOutcome.isnull()] = mean_age
    
    # One-Hot Encode type, sex, breed and color
    df['SexuponOutcome'][df['SexuponOutcome'].isnull()] = 'Unknown'
    df['Color'] = df['Color'].apply(simplifyColor)
    df['Breed'] = df['Breed'].apply(simplifyBreed)
    if hotEncode:
        breed = pd.get_dummies(df['Breed'])
        color = pd.get_dummies(df['Color'])
    else:
        animalType = df['AnimalType']
        breed = df['Breed']
        color = df['Color']
        
    mix = df['Breed'].copy().apply(isMix)
    spayed = df['SexuponOutcome'].apply(isSpayedNeutered)
    


    return pd.concat([hasname, nameslength, year, month, day, weekday, ages, spayed,
                      breed, mix, color], axis=1)

def encodeLabels(df, hotEncode):
    return df['OutcomeType']



encode = True

X_cat = encodeFeatures(train[train['AnimalType'] == "Cat"], encode)
y_cat = encodeLabels(train[train['AnimalType'] == "Cat"], encode)
X_dog = encodeFeatures(train[train['AnimalType'] == "Dog"], encode)
y_dog = encodeLabels(train[train['AnimalType'] == "Dog"], encode)
print(X_cat.shape)
print(X_dog.shape)

(11134, 83)
(15595, 370)


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

def getModel(X, y):
    X = np.array(X)
    y = np.array(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=0)
    # # Feature Scaling
    # if encode:
    #     sc = StandardScaler()
    #     X_train = sc.fit_transform(X_train)
    #     X_test = sc.transform(X_test)

    clf = RandomForestClassifier(n_estimators=50)

    clf.fit(X_train,y_train)

    y_pred = np.array(clf.predict_proba(X_test))
    # Model Accuracy, how often is the classifier correct?
    print("Log-loss:",metrics.log_loss(y_test, y_pred))
    return clf

cat_clf = getModel(X_cat, y_cat)
dog_clf = getModel(X_dog, y_dog)

Log-loss: 0.9886145930559224
Log-loss: 1.1548602534915637


In [5]:
test_dog = test[test.AnimalType == "Dog"]
test_cat = test[test.AnimalType == "Cat"]

test_features_dog = encodeFeatures(test_dog.drop(["ID"], axis='columns', inplace=False), encode)
test_features_cat = encodeFeatures(test_cat.drop(["ID"], axis='columns', inplace=False), encode)

# אולי חסרות עמודות לאנקודינג לעומת הסדרת אימון. מה לעשות?

test_pred[test.AnimalType == "Dog"] = dog_clf.predict_proba(test_features_dog)
test_pred[test.AnimalType == "Cat"] = dog_clf.predict_proba(test_features_cat)

prediction = pd.DataFrame(test_pred)
prediction.columns = ['Adoption', 'Died', 'Euthanasia', 'Return_to_owner', 'Transfer']
prediction = pd.concat([test['ID'], prediction], axis=1)
prediction.to_csv('resuts.csv', index=False)

ValueError: Number of features of the model must match the input. Model n_features is 370 and input n_features is 328 